<a href="https://colab.research.google.com/github/heitor-eng/H_Data_Science/blob/master/Web_Scraping_da_cota%C3%A7%C3%A3o_do_Leite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extração da tabela da cotação do leite de site.

Esse é um trabalho inspirado em uma das tarefas de um processo de seletivo em que participei. O processo seletivo em si era somente a extração de um dado específico de um site qualquer, porém nesse trabalho resolvi ao invés de importar somente o dado, porque não importar uma tabela ou tabelas e fazer um tratamento nos dados. O passo a passo da tarefa que será apresentada aqui:

* Script em Python para extrair a tabela com o preço do leite do website: [Notícias Agrícolas](https://www.noticiasagricolas.com.br/cotacoes/leite/leite-precos-ao-produtor-cepea-rs-litro),

* Salvar os arquivos em `.xlsx` ou em `.csv`,

* Tratamento dos dados na tabela.



## Bibliotecas Utilizadas:

* `requests`: biblioteca para execução das requisições HTTP;
* `BeautifulSoup`: biblioteca para a extração de dados em arquivos HTMLe XML;
* `Pandas`: biblioteca para armazenar, limpar e salvar os dados em tabela.

Procedimento de trabalho: a biblioteca `request` faz o processo de executar requesições GET e obter o código HTML da página, com isso feito utilizo a `BeautifulSoup` para extrair os dados e por último usar o `Pandas` para trabalhar com esses dados em forma de DataFrame.

In [ ]:
#importando as bibliotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup

Nesta  etapa, o procedimento é fazer uma requisição GET da página, consigo fazer isso chamando o método `requests.get` com a URL da pagina como argumento.
Esse método retorna um objeto Response que contém várias informações das quais utilizo o `status_code` para verificar se a requisição retorna um status 200, indicando que foi bem sucedida, e o `content` para acessar o código da página HTML. 

In [ ]:
# Execução do Request Get para obter o conteúdo da pasta 
req = requests.get('https://www.noticiasagricolas.com.br/cotacoes/leite/leite-precos-ao-produtor-cepea-rs-litro')
if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content

Requisição bem sucedida!


Com o HTML da página agora consigo usar a biblioteca `BeautifulSoup` para extrair a tabela. Feito isso é necessário criar um objeto que irá salvar o documento de maneira estruturada de acordo com as tags e depois posso acessar o elemento necessário utilizando `find` ou `find_all`, passando como argumento o nome da tag que neste caso é `table`.

In [ ]:
# objeto BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# método find_all
table = soup.find_all(name='table')


Com o código HTML, das tabelas em mãos, chamo a biblioteca `Pandas` para carregar esses dados em um DataFrame através do método `read_html` e assim tenho o DataFrame das tabelas. Porém, primeiro tenho que converter a variável `table` em uma *string* , pois até agora é um **objeto** do `BeautifulSoup`, e com isso feito agora eu tenho uma lista de Data Frames, ou seja, cada tabela que está nesse site virou uma lista de DataFrames isso é uma resposta do método `BeautifulSoup` para essa extração e portanto se acessar a posição 0, ou seja, se quiser obter somente um tabela é necessário digitar `df[0]` ao invés de `df` ou pode usar o método `find()` ao invés do `find_all()`.

In [ ]:
# lendo como um DataFrame
table_str = str(table)
df = pd.read_html(table_str)

df[0]


,Estados,Preço (R$/Litro),Variação (%)
0,MG,20550.0,"+2,37"
1,RS,19413.0,"+1,47"
2,SP,20438.0,"+2,85"
3,PR,19915.0,"+3,68"
4,GO,20651.0,"+4,20"
5,BA,18802.0,-008
6,SC,20218.0,"+3,07"
7,MS,17641.0,"+4,34"
8,ES,19084.0,"+2,29"
9,Brasil,20364.0,"+2,66"


Com isso feito a gente já só precisa exportar essa lista de Data Frames, para os formato `xlsx` e em `csv`. Fazendo isso a gente consegue trabalhar no *dataset*, como é mostrado no exemplo a seguir. 

In [ ]:
# exportando a primeira tabela para o excel
df[0].to_excel('arquivo1.xlsx')

In [ ]:
# exportando para csv
df[0].to_csv("Tabela1.csv")

In [9]:
df = pd.read_csv("/content/Tabela1.csv")
df

,Unnamed: 0,Estados,Preço (R$/Litro),Variação (%)
0,0,MG,20550.0,"+2,37"
1,1,RS,19413.0,"+1,47"
2,2,SP,20438.0,"+2,85"
3,3,PR,19915.0,"+3,68"
4,4,GO,20651.0,"+4,20"
5,5,BA,18802.0,-008
6,6,SC,20218.0,"+3,07"
7,7,MS,17641.0,"+4,34"
8,8,ES,19084.0,"+2,29"
9,9,Brasil,20364.0,"+2,66"


Como é possível verificar temos 3 problemas nesse DataFrame, o primeiro é a coluna **Unnamed = 0** que nada mais é do que a coluna índice do DataFrame, depois remover a linha 10 e por último arrumar as grandezas dos preços da coluna **Preço (R$/Litro)** porque o preço do leite não veio com os pontos ou vírgulas no lugar certo nos informando uma ordem de preço errado. Esses serão os 3 tratamentos vão ser feitos nessa tabela.

In [19]:
# tirando a coluna índice
df_1 = pd.read_csv("/content/Tabela1.csv",index_col=0)

# removendo a linha 10
df_1 = df_1.drop([10])

# dividindo a coluna Preço por 10000 para ter o verdadeiro valor correto
df_1["Preço (R$/Litro)"] = df_1["Preço (R$/Litro)"].apply(lambda x:x / 10000)
df_1

df_1

,Estados,Preço (R$/Litro),Variação (%)
0,MG,2.0550,"+2,37"
1,RS,1.9413,"+1,47"
2,SP,2.0438,"+2,85"
3,PR,1.9915,"+3,68"
4,GO,2.0651,"+4,20"
5,BA,1.8802,-008
6,SC,2.0218,"+3,07"
7,MS,1.7641,"+4,34"
8,ES,1.9084,"+2,29"
9,Brasil,2.0364,"+2,66"


Com isso temos o tratamento dos dados do DataFrame feito e pronto para montar gráficos ou analises.